In [6]:
%config IPCompleter.greedy=True

In [7]:
import sys
import os
import pandas as pd
import cv2
import csv
sys.path.append("../..") # relative path to module toolkit

In [8]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.backend import clear_session
import numpy as np
import matplotlib.pyplot as plt
from toolkit import getLabelsFromDir
%matplotlib inline

In [9]:
target_size = (416, 416)
genus_model = load_model("./highest_val_acc.h5")

In [10]:
import os

genus_val_dir = "../../../images/images_genus_shuffled_aspect_416/val/"

genus_labels = getLabelsFromDir(genus_val_dir)
all_genus_predict = []

for cutout_species in [
    "Bisnius nigriventris",
    "Gabrius piliger",
    "Lathrobium castaneipenne",
    "Lathrobium longulum",
    "Lathrobium pallidipenne",
    "Neobisnius procerulus",
    "Philonthus pseudovarians",
    "Philonthus spinipes",
    "Philonthus umbratilis",
    "Platydracus chalcocephalus",
    "Quedius invreae",
    "Quedius nemoralis",
    "Rugilus geniculatus",
    "Rugilus similis",
    "Scopaeus minimus",
    "Tasgius pedator",
    "Tasgius winkleri",
    "Tetartopeus rufonitidus"
    ]:
    
    cutout_species_dir_path = "../../../images/cutout_416_aspect_images_species/val/" + cutout_species
    genus_predicts = []
    
    for file in os.listdir(cutout_species_dir_path):
        if file.endswith(".jpg"):
            file_path = os.path.join(cutout_species_dir_path, file)

            real_genus = file_path.split(" ")[0].split("/")[-1]
            real_species = file_path.split("/")[-1].split("_")[0]

            img = cv2.imread(file_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            resized_img = cv2.resize(img, target_size) 

            img_array = img_to_array(resized_img) / 255.0
            img_array = img_array.reshape((1,) + img_array.shape)
            genus_Y_pred = genus_model.predict(img_array, batch_size=None, verbose=0, steps=None)
            genus_y_pred = np.argmax(genus_Y_pred, axis=1)

            predicted_genus = genus_labels[genus_y_pred[0]]
            
            obj = {"correct_prediction": predicted_genus == real_genus, "filepath": file_path, "real_genus": real_genus, "real_species": real_species, "predicted_genus": predicted_genus}
            
            genus_predicts.append(obj)
            all_genus_predict.append(obj)
            
    keys = genus_predicts[0].keys()
    with open('./cutout/' + cutout_species + '_results.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(genus_predicts)

        
keys = all_genus_predict[0].keys()
print(keys)
with open('./cutout/overall_results.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_genus_predict)
        
        
        

dict_keys(['correct_prediction', 'predicted_genus', 'real_species', 'real_genus', 'filepath'])
